In [1]:
import datetime
import shutil
import time
from pathlib import Path

import numpy as np
import torch
from PIL import Image
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import (resnet18, resnet34, resnet50, resnet101,
                                resnet152, resnext50_32x4d, resnext101_32x8d,
                                wide_resnet50_2, wide_resnet101_2)


import datetime
import time
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from torch import nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50, vgg16, vgg16_bn

from tqdm import tqdm

In [2]:
# Hyperparameters
initial_lr = 0.001
num_epochs = 30
batch_size = 64

# Data
train_img_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\train')
test_img_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\test')

device = torch.device("cuda:0")

In [3]:
class Trainer:

    def __init__(self, model) -> None:
        self.model = model.to(device)
        self.optimizer = Adam(self.model.parameters(), lr=initial_lr)
        self.loss_func = nn.CrossEntropyLoss().to(device)
        self.scheduler = StepLR(self.optimizer, step_size=7, gamma=0.4)

        self.total_time = 0
        self.train_loader = None
        self.test_loader = None
        self.labels = None

        self.create_output_dir()
        self.load_data()

    def create_output_dir(self):
        # 获取当前时间的字符串表示形式
        current_time = datetime.datetime.now().strftime(f"%Y-%m-%d_%H-%M-%S")
        # 生成以当前时间命名的目录路径
        self.output_dir = Path.cwd().parent / 'output' / 'run' / current_time
        # 创建目录
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def load_data(self):
        # 加载训练集
        transform_train = transforms.Compose([
            transforms.Grayscale(1),
            transforms.RandomHorizontalFlip(),  # 随机水平翻转
            transforms.RandomVerticalFlip(),  # 随机垂直翻转
            transforms.RandomRotation(360),  # 随机旋转
            transforms.ToTensor()
        ])
        train_data = datasets.ImageFolder(root=train_img_path.as_posix(), transform=transform_train)
        self.train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)

        # 加载测试集
        transform_test = transforms.Compose([transforms.Grayscale(1), transforms.ToTensor()])
        test_data = datasets.ImageFolder(root=test_img_path.as_posix(), transform=transform_test)
        self.test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True)

        self.labels = [x for x, _ in sorted(test_data.class_to_idx.items(), key=lambda x: x[1])]

    def train(self):
        self.model.train()
        running_loss = 0
        for inputs, labels in self.train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = self.model(inputs)
            self.optimizer.zero_grad()
            loss = self.loss_func(outputs, labels)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        self.scheduler.step()

        return running_loss / len(self.train_loader.dataset)

    def validate(self):
        self.model.eval()
        running_loss = 0
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = self.model(inputs)
                loss = self.loss_func(outputs, labels)
                running_loss += loss.item() * inputs.size(0)

        return running_loss / len(self.test_loader.dataset)

    def run(self):
        with open(self.output_dir / 'log.txt', 'w') as log:
            log.write(f'Number of epochs: {num_epochs}\n')
            log.write(f'Initial learning rate: {initial_lr}\n')
            log.write(f'Batch size: {batch_size}\n')
            log.write(f'Optimizer: Adam\n')
            log.write(f'Scheduler: StepLR\n')
            log.write(f'Loss function: CE\n')
            log.write(f'Model: {self.model}\n')
            log.write(f'Device: {device}\n\n')

            with tqdm(range(num_epochs), desc='Progress') as tbar:
                for epoch in range(num_epochs):
                    epoch_start = time.time()
                    train_loss = self.train()
                    valid_loss = self.validate()
                    time_elapsed = time.time() - epoch_start

                    log.write(f'Epoch {epoch+1:02d}/{num_epochs}, ')
                    log.write(f'Training Loss: {train_loss:.4f}, ')
                    log.write(f'Validation Loss: {valid_loss:.4f}, ')
                    log.write(f'Time: {time_elapsed:.2f}s\n')
                    log.flush()
                    tbar.update()

            torch.save(self.model, self.output_dir / f'model.pth')

            self.eval_accuracy()
            self.plot_confusion_matrix()

    def eval_accuracy(self):
        # 精度评估
        self.model.eval()

        all_labels = torch.tensor([]).to(device)
        all_predictions = torch.tensor([]).to(device)

        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = self.model(inputs)
                outputs = torch.argmax(outputs, dim=1)

                all_labels = torch.cat((all_labels, labels), dim=0)
                all_predictions = torch.cat((all_predictions, outputs), dim=0)

        # 生成混淆矩阵
        self.cm = confusion_matrix(all_labels.cpu(), all_predictions.cpu())
        # 计算精度
        # self.accuracy = metrics.accuracy_score(all_labels.cpu(), all_predictions.cpu())
        self.accuracy = (all_labels - all_predictions).abs().mean().item()

    def plot_confusion_matrix(self):
        # 画混淆矩阵图
        _, ax = plt.subplots(figsize=(5, 5), dpi=300)
        ax.matshow(self.cm, cmap=plt.cm.Reds)

        for i in range(len(self.cm)):
            for j in range(len(self.cm)):
                plt.annotate(self.cm[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center')

        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.xticks(range(28), labels=self.labels)
        plt.yticks(range(28), labels=self.labels)
        plt.tight_layout()

        pic_path = self.output_dir / 'confusion_matrix.png'
        plt.savefig(pic_path.as_posix())
        plt.close()

In [5]:
torch.cuda.empty_cache()
# 因为任务是回归，所以将最后一层的输出维度设置为1
model = resnet50(weights=None, num_classes=28)
# 重置第一层卷积层，将输入由三通道变为单通道
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

trainer = Trainer(model)
trainer.run()

Progress: 100%|██████████| 30/30 [14:02<00:00, 28.07s/it]


In [4]:
# 因为任务是回归，所以将最后一层的输出维度设置为1
model = vgg16_bn(weights=None, num_classes=28)
# 重置第一层卷积层，将输入由三通道变为单通道
model.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)

trainer = Trainer(model)
trainer.run()

Progress: 100%|██████████| 30/30 [30:28<00:00, 60.94s/it]


In [ ]:
# 精度评估
difference = []
weight = []
losses = []

trainer.model.eval()
with torch.no_grad():
    for inputs, labels in trainer.test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = trainer.model(inputs)
        # 四舍五入到最近的整数, 将所有小于1的值设为1
        # outputs = outputs.round().clamp(min=1)
        preds = torch.argmax(outputs, dim=1).float()

        difference.append((preds - labels).abs().mean().item())
        weight.append(inputs.size(0))

# 计算精度
overall_acc = np.average(difference)

In [ ]:
overall_acc

In [ ]:
trainer.model.eval()

all_labels = torch.tensor([]).to(device)
all_predictions = torch.tensor([]).to(device)

with torch.no_grad():
    for inputs, labels in trainer.test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = trainer.model(inputs)
        outputs = torch.argmax(outputs, dim=1)

        all_labels = torch.cat((all_labels, labels), dim=0)
        all_predictions = torch.cat((all_predictions, outputs), dim=0)

# 生成混淆矩阵
trainer.cm = confusion_matrix(all_labels.cpu(), all_predictions.cpu())

# 画混淆矩阵图
_, ax = plt.subplots(figsize=(10,10), dpi=300)
ax.matshow(trainer.cm, cmap=plt.cm.Reds)

for i in range(len(trainer.cm)):
    for j in range(len(trainer.cm)):
        plt.annotate(trainer.cm[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.xticks(range(28), labels=trainer.labels)
plt.yticks(range(28), labels=trainer.labels)
plt.tight_layout()

pic_path = trainer.output_dir / 'confusion_matrix.png'
plt.savefig(pic_path.as_posix())
plt.show()